In [1]:
from sys import path
from os.path import relpath, realpath, abspath, join, exists
from itertools import product

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

pd.set_option('display.max_rows', 999)

In [2]:
from torchnetjson.builder import build_net

In [3]:
from thesis_v2 import dir_dict
from thesis_v2.data import load_data_helper
from thesis_v2.training.training_aux import load_training_results
from thesis_v2.training_extra.misc import count_params
from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import load_modules

load_modules()

In [4]:
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200208 as param_iterator_obj,
    keygen
)

In [5]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [19]:
def load_everything():
    _df_collect = []
    for idx, param in enumerate(param_iterator_obj().generate()):
        if idx % 100 == 0:
            print(idx)
            
#         if idx >= 100:
#             break
            
#         print(len(param))
        
        assert len(param) == 19
        
        assert param['split_seed'] == 'legacy'
        assert param['pooling_ksize'] == 3
        assert param['pooling_type'] == 'avg'
        assert not param['bn_after_fc']
        assert param['rcnn_bl_psize'] == 1
        assert param['rcnn_bl_ptype'] is None
        assert param['input_size'] == 50
        # 12 to go
        # remove smoothness_name and scale_name, 10 to go
        
#         assert param['model_seed'] == 0
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        # 10 to go.
        result = load_training_results(key, return_model=False)
        # load twice, first time to get the model.
#         result = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))
        num_epochs = [len(x) for x in result['stats_all']]
        
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
        
        _df_collect.append(
            {
                'act_fn': param['act_fn'],
                'loss_type': param['loss_type'],
                'scale': float(param['scale']),
                'smoothness': float(param['smoothness']),
                'num_layer': param['num_layer'],
                'out_channel': param['out_channel'],
                'kernel_size_l1': param['kernel_size_l1'],
                # 4 to go.

                'rcnn_bl_cls': param['rcnn_bl_cls'],
                'rcnn_acc_type': param['rcnn_acc_type'],

                'model_seed': param['model_seed'],
                # done,
                'corr_test': ((cc_raw/cc_max_all_neurons)**2).mean(),
                'max_epoch': max(num_epochs),
#                 'num_param': count_params(result['model']),
            }
        )
    _df =  pd.DataFrame(_df_collect, columns=[
#             'split_seed',
        'act_fn',
        'loss_type', 'scale', 'smoothness', 'num_layer',
        'kernel_size_l1', 'out_channel',

        'rcnn_bl_cls', 'rcnn_acc_type', 'model_seed',

        'corr_test',
#         'num_param',
        'max_epoch',
    ])
#     print(_df)
    _df = _df.set_index([
        'act_fn',
        'loss_type', 'scale', 'smoothness', 'num_layer',
        'kernel_size_l1', 'out_channel',
        'rcnn_bl_cls', 'rcnn_acc_type', 'model_seed',
    ], verify_integrity=True)
    return _df

In [20]:
df = load_everything()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [21]:
corr_mean = df['corr_test'].unstack('model_seed').mean(axis=1)
corr_std = df['corr_test'].unstack('model_seed').std(axis=1)

In [22]:
corr_mean.unstack('rcnn_bl_cls')

rcnn_bl_cls                                                                                    1  \
act_fn   loss_type scale smoothness num_layer kernel_size_l1 out_channel rcnn_acc_type             
relu     mse       0.01  0.000005   2         7              16          cummean        0.635999   
                                                                         instant        0.635999   
                                                             48          cummean        0.657493   
                                                                         instant        0.656403   
                                              9              16          cummean        0.623429   
                                                                         instant        0.623429   
                                                             48          cummean        0.641707   
                                                                         instant        0.640790   
                                              13             16          cummean        0.600643   
                                                                         instant        0.599279   
                                                             48          cummean        0.615156   
                                                                         instant        0.618492   
                                    3         7              16          cummean        0.626239   
                                                                         instant        0.630375   
                                                             48          cummean        0.661742   
                                                                         instant        0.661106   
                                              9              16          cummean        0.622675   
                                                                         instant        0.625014   
                                                             48          cummean        0.644304   
                                                                         instant        0.644304   
                                              13             16          cummean        0.607852   
                                                                         instant        0.606006   
                                                             48          cummean        0.622476   
                                                                         instant        0.618627   
                         0.005000   2         7              16          cummean        0.439788   
                                                                         instant        0.440008   
                                                             48          cummean        0.470165   
                                                                         instant        0.469985   
                                              9              16          cummean        0.423042   
                                                                         instant        0.417501   
                                                             48          cummean        0.429657   
                                                                         instant        0.430055   
                                              13             16          cummean        0.403771   
                                                                         instant        0.403771   
                                                             48          cummean        0.410301   
                                                                         instant        0.410587   
                                    3         7              16          cummean        0.482668   
                                                                         instant        0.485989   


In [23]:
corr_std.unstack('rcnn_bl_cls')

rcnn_bl_cls                                                                                    1  \
act_fn   loss_type scale smoothness num_layer kernel_size_l1 out_channel rcnn_acc_type             
relu     mse       0.01  0.000005   2         7              16          cummean        0.006403   
                                                                         instant        0.006403   
                                                             48          cummean        0.000923   
                                                                         instant        0.002464   
                                              9              16          cummean        0.007016   
                                                                         instant        0.007016   
                                                             48          cummean        0.004190   
                                                                         instant        0.005486   
                                              13             16          cummean        0.000924   
                                                                         instant        0.001005   
                                                             48          cummean        0.001217   
                                                                         instant        0.005936   
                                    3         7              16          cummean        0.006585   
                                                                         instant        0.000301   
                                                             48          cummean        0.002282   
                                                                         instant        0.005867   
                                              9              16          cummean        0.000282   
                                                                         instant        0.003589   
                                                             48          cummean        0.001259   
                                                                         instant        0.001259   
                                              13             16          cummean        0.005037   
                                                                         instant        0.007238   
                                                             48          cummean        0.002597   
                                                                         instant        0.008040   
                         0.005000   2         7              16          cummean        0.006841   
                                                                         instant        0.006530   
                                                             48          cummean        0.000713   
                                                                         instant        0.000966   
                                              9              16          cummean        0.007634   
                                                                         instant        0.000203   
                                                             48          cummean        0.015881   
                                                                         instant        0.015586   
                                              13             16          cummean        0.001923   
                                                                         instant        0.001923   
                                                             48          cummean        0.001681   
                                                                         instant        0.002086   
                                    3         7              16          cummean        0.006437   
                                                                         instant        0.004191   
